uses cols:['id','click','hour','banner_pos','device_id','device_ip', 
'device_model','device_conn_type','C14','C17','C20','C21'],

and add cols ['pub_id','pub_domain','pub_category','device_id_count','device_ip_count','user_count','smooth_user_hour_count','user_click_histroy'].

Split rows into site rows and app rows.
Apply model to each.
A row is app iff site_id == 85f751fd.

Result:
- categorical_features = ['banner_pos', 'site_id', 'site_category',
        'app_id', 'app_category', 'user',
                        'device_conn_type','C14','C17','C20','C21'],
  counts: device_id/ip, user, hourly_user
  => bad. score ~= -50.


In [1]:
import pandas as pd
import numpy as np

from models.base import create_user, site_app_split
from models.model_four import tune_model_four

In [2]:
df = pd.read_csv('data/train_tiny.csv')
df.hour = pd.to_datetime(df.hour, format="%y%m%d%H")
df = create_user(df)
df_site, df_app = site_app_split(df)

## Stats
- app:
            {'logistic_regression__C': 0.01},
            {0.0001: -0.455594184017866,
            0.00025118864315095795: -0.45558380267601795,
            0.000630957344480193: -0.4555796421227544,
            0.001584893192461114: -0.4555779813755934,
            0.003981071705534973: -0.4555773195343969,
            0.01: -0.4555770559619175},
            -0.4715479739114517
            
- site:
            {'logistic_regression__C': 0.001},
            {0.001: -0.5220806343621686,
            0.31622776601683794: -0.5220806343621966,
            100.0: -0.5220806343622033},
            -0.5128507326797699

In [3]:
params = np.logspace(-5, 2, num=4)
params

array([1.00000000e-05, 2.15443469e-03, 4.64158883e-01, 1.00000000e+02])

In [4]:
# fit on app
best_C, params_dict_ls, scores, test_score = tune_model_four(df_app, params)
best_C, dict(zip(params, scores)), test_score

Tuning time:  1.6564240455627441


({'logistic_regression__C': 0.4641588833612782},
 {1e-05: -0.45185333872823275,
  0.0021544346900318843: -0.4033415675673009,
  0.4641588833612782: -0.3083780412590003,
  100.0: -0.4479216127508794},
 -0.4655719930304711)

In [5]:
params = np.logspace(-3, 2, num=3)
params

array([1.00000000e-03, 3.16227766e-01, 1.00000000e+02])

In [6]:
# fit on site
best_C, params_dict_ls, scores, test_score = tune_model_four(df_site, params)
best_C, dict(zip(params, scores)), test_score

Tuning time:  1.1619603633880615


({'logistic_regression__C': 0.001},
 {0.001: -0.4987297020872866,
  0.31622776601683794: -0.6497838629435246,
  100.0: -1.6085005162487989},
 -0.4683502561590366)

In [7]:
raise

RuntimeError: No active exception to reraise

## Stats: without splitting
- {'logistic_regression__C': 1e-05},
            [{'logistic_regression__C': 1e-05},
            {'logistic_regression__C': 0.0021544346900318843},
            {'logistic_regression__C': 0.4641588833612782},
            {'logistic_regression__C': 100.0}],
            [-0.5079341890609894,
            -0.5079341890610994,
            -0.5079341890610902,
            -0.507934189061084],
            -0.4957722697557072

In [ ]:
df = pd.read_csv('data/train_tiny.csv')
df.hour = pd.to_datetime(df.hour, format="%y%m%d%H")
df = create_user(df)

In [ ]:
params = np.logspace(-5, 2, num=4)
params

In [ ]:
best_C, params_dict_ls, scores, test_score = tune_model_four(df, params)
best_C, dict(zip(params, scores)), test_score

In [ ]:
raise

## Develop pipeline

In [ ]:
pipeline = get_model_four_pipeline()

In [ ]:
pipeline.fit(df, df.click)

In [ ]:
pipeline.predict_proba(df)

## Develop HourlyCountEncoder

In [ ]:
counts = df.groupby([df.hour, 'user']).size().to_frame('count')
pd.merge(df, counts, how='left', on=['hour', 'user']).head()

## Develop CountEncoder

In [ ]:
counts = df.groupby('device_ip').size().to_frame('device_ip_count')

In [ ]:
pd.merge(df, counts, how='left', on='device_ip')

In [ ]:
df['device_ip_count'] = 

## Define user
user := ip & model if device_id is null (== a99f214a), else device_id.

In [ ]:
user_cols = ['device_ip', 'device_model', 'device_id']

In [ ]:
device_id_null = 'a99f214a'
(df.device_id == device_id_null).sum()

In [ ]:
df['device_ip_model'] = df.device_ip.str.cat(df.device_model, sep='_')

In [ ]:
df[user_cols + ['device_ip_model']].head()

In [ ]:
df['user'] = df.device_id.where(df.device_id != device_id_null, df.device_ip_model)

In [ ]:
df[user_cols + ['device_ip_model', 'user']].head()

In [ ]:
df[df.device_id != device_id_null][user_cols + ['device_ip_model', 'user']].head()